# NasNet Dog breed classification

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.conv_learner import *
PATH = "data/dogbreed/"
sz=224; bs=48

In [3]:
def nasnet(pre): return nasnetalarge(pretrained = 'imagenet' if pre else None)
model_features[nasnet]=4032*2

In [4]:
label_csv = f'{PATH}labels.csv'
n = len(list(open(label_csv))) - 1
val_idxs = get_cv_idxs(n)

In [5]:
stats = ([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
tfms = tfms_from_stats(stats, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
#data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs)

#tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_csv(PATH, 'train', f'{PATH}labels.csv', test_name='test',
                                    val_idxs=val_idxs, suffix='.jpg', tfms=tfms, bs=bs)

In [6]:
learn = ConvLearner.pretrained(nasnet, data, precompute=True, xtra_fc=[], ps=0.5)

In [7]:
%time learn.fit(1e-2, 5)

A Jupyter Widget

[ 0.       0.73129  0.57436  0.88656]                        
[ 1.       0.40453  0.55798  0.88704]                         
[ 2.       0.27911  0.60471  0.88545]                         
[ 3.       0.2345   0.59764  0.88912]                         
[ 4.       0.18217  0.61055  0.88947]                         

CPU times: user 5.4 s, sys: 4.46 s, total: 9.86 s
Wall time: 14.7 s


In [8]:
learn.precompute=False
learn.bn_freeze=True

In [9]:
%time learn.fit(1e-2, 3, cycle_len=1)

A Jupyter Widget

[ 0.       0.29864  0.54605  0.89223]                        
[ 1.       0.25673  0.53777  0.89563]                        
[ 2.       0.24703  0.54247  0.89223]                        

CPU times: user 2h 50min 36s, sys: 17min 10s, total: 3h 7min 47s
Wall time: 3h 12min 17s


In [ ]:
%time learn.fit(1e-2, 3, cycle_len=1, cycle_mult=2)

A Jupyter Widget

 50%|████▉     | 85/171 [25:36<25:54, 18.07s/it, loss=0.239] 

In [10]:
learn.save('nas_pre')

In [11]:
def freeze_to(m, n):
    c=children(m[0])
    for l in c:     set_trainable(l, False)
    for l in c[n:]: set_trainable(l, True)

In [12]:
freeze_to(learn.model, 17)

In [13]:
learn.fit([1e-5,1e-4,1e-2], 1, cycle_len=1)

A Jupyter Widget

[ 0.       0.40278  0.47714  0.89078]                           



In [14]:
learn.save('nas')